# メルセンヌ素数

In [1]:
import math
import matplotlib.pyplot as plt
%matplotlib inline
from sympy import *
from tqdm import tqdm_notebook as tqdm
from decimal import *
getcontext()
from functools import reduce

## メルセンヌ素数を探す

まずは素数判定関数を定義

In [45]:
def find_a_factor(num):
    for k in range(2, int(math.sqrt(num))+1):
        if num%k == 0:
            return k
            break
    else: return 0
    
def is_prime(num):
    return find_a_factor(num)==0

素数リスト `plist` の作成

In [17]:
Num = 60
plist = [n for n in tqdm(range(2,Num+1)) if is_prime(n)]
print(plist)


[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59]


メルセンヌ素数を素朴に探す。

In [ ]:
for k in tqdm(plist):
    if is_prime(k): print('2^{}-1 = {} is prime'.format(k, 2**k-1))

$2^{61}-1$ の素数判定はずいぶん時間がかかる。

In [ ]:
def find_factor_w_tqdm(num):
    for k in tqdm(range(2, int(math.sqrt(num))+1)):
        if num%k == 0:
            return k
            break
    else: return 0

In [ ]:
result = find_factor_w_tqdm(2**61-1); print(result)

## フェルマーテスト

素数判定をもっと高速のものにしたい。まずはフェルマーテスト。

In [ ]:
def fermat(num):
    num = abs(num)
    if num == 2:
        return True
    elif num < 2 or num%2 == 0:
        return False
    else:
        return pow(2, num-1, num) == 1

In [ ]:
for k in tqdm(plist):
    if fermat(2**k-1): print('2^{}-1 = {} is prime'.format(k, 2**k-1))

上の出力のように、$2^{11}-1$ や $2^{23}-1$ が素数と判定されてしまった。

## リュカ-レーマー・テスト

#### 疑似コード(from Wikipedia)
```
入力: p:奇素数であるテスト対象の整数
出力: PRIME:素数の場合, COMPOSIT:合成数の場合
Lucas_Lehmer_Test(p):
    var s = 4
    var M = 2**p − 1
    for n in range(2, p):
        s = (s*s-2)%M
    if s == 0 then
        return PRIME
    else
        return COMPOSIT
```

In [38]:
def lucas_lehmer(p):
    s = 4
    mp = 2**p-1
    for n in range(p-2): # p-2 times iteration (list(range(p-2))=[0,1,...,p-1])
         s = (s**2-2)%mp
    return s==0

In [26]:
def lucas_lehmer_FAST(p):
    s = 4
    mp = (1<<p)-1
    for n in range(p-2):
        ss = s*s        
        s = (ss & mp) + (ss >> p)
        if s >= mp: s = s-mp
        s = s-2
    return s==0

In [ ]:
%timeit lucas_lehmer(4423)
%timeit lucas_lehmer_FAST(4423)

### リュカ-レーマー・テストを使う。

In [ ]:
def mp_str(p):
    mp = Decimal(2**p-1)
    nod = mp.adjusted()+1
    if nod <= 40:
        return str(mp)
    else:
        getcontext().prec = nod
        top = (mp/(Decimal(10)**(nod-20))).quantize(1)
        bottom = mp-math.floor(mp/Decimal(1.0e+20))*Decimal(1.0e+20)
        return '{}...{} [{} digits]'.format(top,bottom,nod)

def find_mp(nmin, nmax):
    plist = [n for n in range(nmin, nmax+1) if is_prime(n)]
    for p in tqdm(plist):
        if lucas_lehmer_FAST(p): print('2^{}-1 = {} is prime'.format(p, mp_str(p)))

In [ ]:
find_mp(100, 1000)

In [ ]:
find_mp(1000, 4000)

In [ ]:
find_mp(10000, 20000)

In [ ]:
find_mp(20000, 50000)

### リュカ-レーマー・テストのコードの改良

$s_{p-2}$ を一般項から直接求める。

In [ ]:
getcontext().prec=100000
p = 17
mp = 2**p-1
seq = [4]
for n in range(p-2): seq.append((seq[-1]**2-2)%mp)
print(seq)

In [ ]:
def binom(n,k,mod):
#    nume = Decimal(math.factorial(n))
#    deno = Decimal(math.factorial(k)*math.factorial(n-k))
    if k==0: nume, deno = 1, 1
    else:
        nume = reduce(lambda x,y: x*y, range(n-k+1, n+1))
        deno = reduce(lambda x,y: x*y, range(1, k+1))
    return Decimal(nume)/Decimal(deno)

In [ ]:
getcontext().prec=10000
p = 17
mp = Decimal(2**p-1)
for i in range(1,p-2+1):
    n = 2**i
    seq = [2*binom(n,2*k,mp)*2**(n-2*k)*3**k for k in range(2**(i-1)+1)]
    print(sum(seq)%mp)

In [ ]:
p = 17
mp = (1<<p)-1
n = 1<<(p-2)
sum([2*binom(n,2*k,mp)*(2**(n-2*k))*(3**k) for k in range(int(n/2)+1)])%mp

### "メルセンヌ合成数" について

In [30]:
def find_a_factor(num):
    num = abs(int(num))
    for n in range(2,int(math.sqrt(num))+1):
        if num%n==0: return n; break
    else: return num
        
def factorisation(num, facs_list=None):
    if facs_list is None: facs_list = []
    #import pdb; pdb.set_trace()
    facs_list.append(find_a_factor(num))
    if facs_list[-1]!=num:
        facs_list=factorisation(int(num/facs_list[-1]), facs_list)
    return facs_list

In [31]:
factorisation(2**23-1)

[47, 178481]

In [48]:
map(list(range(3, 1000)))
    if is_prime(n): plist.append(n)

for p in plist:
    mp = (1<<p)-1
    if not lucas_lehmer_FAST(p):
        factorised = factorisation(mp)
        print("2**{}-1={}={}".format(p,mp,factorised))

[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997]


ZeroDivisionError: division by zero

In [29]:
nn = 2**23-1
factorisation(nn)

ZeroDivisionError: division by zero

## ミラーラビン法

確率的判定法。

In [ ]:
import random
def millerrabin(n):
    if n%2 == 0 or n%3 == 0 or n%5 == 0:
        return False
    else:
        s, d = 0, n-1
        while d%2==0: s,d = s+1, int(d/2)
        k = 50
#        for j in tqdm(range(k)):
        while k > 0:
            k = k-1
            a = random.randint(1,n-1)
            t, y = d, pow(a,d,n)
            while t != n-1 and y != 1 and y != n-1:
                y = pow(y,2,n)
                t <<= 1
            if y != n-1 and t%2 == 0:
                return False
        return True

In [ ]:
for k in tqdm(plist):
    if millerrabin(2**k-1): print('2^{}-1 = {} is prime'.format(k, 2**k-1))

ずいぶん時間がかかってしまう。なぜ?